In [23]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import sys 
sys.path.insert(0, '..')
import kaolin as kal
from kaolin.datasets import shapenet
from kaolin import rep
from kaolin import conversions
import torch
from torch import nn
import matplotlib.pyplot as plt
import math
from models import periodic_shape_sampler_xyz
from models import super_shape_sampler
from models import super_shape
from models import model_utils
import utils
from losses import custom_chamfer_loss
import numpy as np
import random
import tqdm
from collections import defaultdict
from torch.autograd import Variable
import torch.optim as optim
import pickle
import os
import dotenv
import plotly.graph_objects as go
from visualize import plot
import trimesh
from metrics import metrics_functions
from external.PyTorchEMD import emd

In [24]:
seed = 0
random.seed(seed)  
np.random.seed(seed)  
# PyTorch のRNGを初期化  
torch.manual_seed(seed)

In [25]:
dotenv.load_dotenv(verbose=True)

category = 'plane'
cache_root = os.getenv('SHAPENET_KAOLIN_CACHE_ROOT')
shapenet_root = os.getenv('SHAPENET_ROOT')
cache_dir = os.path.join(cache_root, category)

categories = [category]


In [26]:
sdf_set = shapenet.ShapeNet_SDF_Points(root=shapenet_root, categories=categories, cache_dir=cache_dir, train=True, split=1.)
point_set = shapenet.ShapeNet_Points(root=shapenet_root, categories=categories, cache_dir=cache_dir, train=True, split=1.)
surface_set = shapenet.ShapeNet_Surface_Meshes(root=shapenet_root, categories=categories, cache_dir=cache_dir, train=True, split=1.)

converting to surface meshes: 100%|██████████| 4045/4045 [00:00<00:00, 32770.09it/s]


In [27]:
m = 4
n = 6
dim = 3
points_sample_num = 3000
sample_idx = 0
train_theta_sample_num = 100

device_type = 'cuda:0'
device = torch.device(device_type)

primitive = super_shape.SuperShapes(m, n, quadrics=True, train_ab=False, dim=dim, transition_range=3)
primitive.to(device)
primitive.eval()
primitive.load_state_dict(torch.load('primitive.pth', map_location=device))

sampler = periodic_shape_sampler_xyz.PeriodicShapeSamplerXYZ(points_sample_num, m, n, factor=2, dim=dim)
sampler.to(device)
sampler.eval()
sampler.load_state_dict(torch.load('periodic_sampler.pth', map_location=device))


<All keys matched successfully>

In [28]:

points = point_set[sample_idx]['data']['points'].to(device) * 10
all_points_num = points.shape[0]

# grid_points_num, dim
xyz = utils.generate_grid_samples(4, sampling='uniform', sample_num=100, dim=dim).to(device)
mesh = surface_set[sample_idx]['data']
meshkal = rep.TriangleMesh.from_tensors(mesh['vertices']*10,
                                    mesh['faces'])
meshkal.to('cuda:0')
sdf_func = kal.conversions.trianglemesh_to_sdf(meshkal, xyz.shape[0])
sgn = (sdf_func(xyz[0].to('cuda:0')).to(device) <= 0.001)


sampled_points = points[random.sample(range(all_points_num), points_sample_num), :].view(1, -1, dim)
print(sampled_points.min()/10, sampled_points.max()/10)
thetas = utils.sample_spherical_angles(batch=1, sample_num=train_theta_sample_num, sampling='uniform', device=device, dim=dim)

pred_points, pred_mask, pred_tsd = sampler(primitive(), points=sampled_points, thetas=thetas, coord=xyz)

pred_surf_points = sampler.extract_super_shapes_surface_point(pred_points, primitive(), points=sampled_points)
print(pred_surf_points.shape, points.shape)
cd1 = metrics_functions.chamfer_distance_l1(pred_surf_points[0].to('cuda:0')[random.sample(range(pred_surf_points.shape[1]), all_points_num), :]/10, points.to('cuda:0')/10)
fscr = kal.metrics.f_score(pred_surf_points[0].to('cuda:0')[random.sample(range(pred_surf_points.shape[1]), all_points_num), :]/10, points.to('cuda:0')/10)

"""
emd_src = pred_surf_points[:, random.sample(range(pred_surf_points.shape[1]), all_points_num), :]/10
emd_dst = points.unsqueeze(0)/10
print(emd_src.shape, emd_dst.shape)
emdscr = emd.earth_mover_distance(emd_src, emd_dst, transpose=False)/all_points_num
"""
print(cd1, fscr)


prd_sgn = model_utils.convert_tsd_range_to_zero_to_one(pred_tsd).sum(1)

tensor(-0.3528, device='cuda:0') tensor(0.3458, device='cuda:0')
torch.Size([1, 52029, 3]) torch.Size([5000, 3])
tensor(0.0164, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8860, device='cuda:0')


In [29]:
prd_sgn_sgn = (prd_sgn >=0.5).view(-1)
iou = ((prd_sgn_sgn & sgn).float().sum() / (prd_sgn_sgn | sgn).float().sum() )
print(iou)

tensor(0.9253, device='cuda:0')


In [31]:
normalizer = kal.transforms.NormalizePointCloud()
normalized_pred_points = normalizer(points/10)
print(normalized_pred_points.min(), normalized_pred_points.max())

tensor(-2.8966, device='cuda:0') tensor(3.3226, device='cuda:0')


In [30]:
bb_min = sampled_points.min(1)[0]
bb_max = sampled_points.max(1)[0]
total_size = (bb_max - bb_min).max()
print(total_size)

tensor(6.9861, device='cuda:0')


In [32]:
bb_min = sampled_points.min(1)[0]
bb_max = sampled_points.max(1)[0]
total_size = (bb_max - bb_min).max()

# Set the center (although this should usually be the origin already).
centers = (bb_min + bb_max) / 2

# Scales all dimensions equally.
scale = total_size

translation = -centers
scale_inv = 1./scale
print(translation, scale_inv)

scaled_points = (sampled_points + translation) * scale_inv
print(scaled_points.min(1)[0], scaled_points.max(1)[0])

tensor([[ 0.0351, -0.0013, -0.0004]], device='cuda:0') tensor(0.1431, device='cuda:0')
tensor([[-0.5000, -0.1600, -0.4775]], device='cuda:0') tensor([[0.5000, 0.1600, 0.4775]], device='cuda:0')


In [33]:
def get_sdf(coord):
    
    coord = coord.unsqueeze(0)
    assert len(coord.shape) == 3
    _, _, tsd = sampler(primitive(), points=sampled_points, coord=coord.to(device)/scale_inv - translation)
    sgn = -((model_utils.convert_tsd_range_to_zero_to_one(tsd).sum(1) >= 0.5).float() - 0.5) 
    return sgn.view(-1).to('cpu')
    
    #return (coord**2).sum(-1).to('cpu') - 0.2
#verts, faces = kal.conversions.sdf_to_trianglemesh(get_sdf, resolution=64)

In [34]:
"""
from visualize import plot
points_list = [pred_points, points.view(1, 1, -1, dim)]
fig = plt.figure()
for idx, points in enumerate(points_list):
    plot.plot_primitive_point_cloud_3d(points)
"""


'\nfrom visualize import plot\npoints_list = [pred_points, points.view(1, 1, -1, dim)]\nfig = plt.figure()\nfor idx, points in enumerate(points_list):\n    plot.plot_primitive_point_cloud_3d(points)\n'

In [35]:
points_from_sdf = (kal.conversions.sdf_to_pointcloud(get_sdf, resolution=64).to(device)/scale_inv-translation)

In [36]:
print(points_from_sdf.shape, translation.device, scale_inv.device, points.shape)

torch.Size([5000, 3]) cuda:0 cuda:0 torch.Size([5000, 3])


In [37]:
cd1 = metrics_functions.chamfer_distance_l1(points_from_sdf/10, points.to(device)/10)
fscr = kal.metrics.f_score(points_from_sdf/10, points.to(device)/10)
print(cd1, fscr)

tensor(0.0186, device='cuda:0') tensor(0.9107, device='cuda:0')


In [38]:
import geomloss
geomloss.SamplesLoss(p=1, blur=1e-7)(points_from_sdf/10, points.to(device)/10)

tensor(0.0090, device='cuda:0')

In [45]:
from pykeops.torch import LazyTensor 
#a = points_from_sdf.view(1, -1, 1, 3)/10
a = torch.zeros(1, 2400, 1, 3).float().to(device)
b = points.to(device).view(1, 1, -1, 3)/10
print(a.shape, b.shape)
al = LazyTensor(a)
bl = LazyTensor(b)
((al - bl) ** 2).norm2().min(2)
((al - bl) ** 2).norm2().min(1).mean()


torch.Size([1, 2400, 1, 3]) torch.Size([1, 1, 5000, 3])


tensor(0.0369, device='cuda:0')

In [40]:
print(pred_points.shape, sampled_points.shape, pred_mask.shape)

torch.Size([1, 6, 10000, 3]) torch.Size([1, 3000, 3]) torch.Size([1, 6, 10000])


In [46]:
from losses import custom_chamfer_loss
custom_chamfer_loss.custom_chamfer_loss(pred_points, sampled_points, surface_mask=pred_mask, pykeops=True)

tensor(0.0020, device='cuda:0', grad_fn=<DivBackward0>)

In [42]:
emdscr = emd.earth_mover_distance(points_from_sdf.unsqueeze(0)/10, points.unsqueeze(0).to(device)/10, transpose=False)/5000*100
print(emdscr)

tensor([0.3449], device='cuda:0')


In [43]:
mesh = trimesh.Trimesh(vertices=verts.cpu().numpy(),
                       faces=faces.cpu().numpy(),
                       process=False)
mesh.show()

NameError: name 'verts' is not defined

In [ ]:
    def refine_mesh(self, mesh, occ_hat, z, c=None):
        ''' Refines the predicted mesh.
        Args:   
            mesh (trimesh object): predicted mesh
            occ_hat (tensor): predicted occupancy grid
            z (tensor): latent code z
            c (tensor): latent conditioned code c
        '''

        self.model.eval()

        # Some shorthands
        n_x, n_y, n_z = occ_hat.shape
        assert(n_x == n_y == n_z)
        # threshold = np.log(self.threshold) - np.log(1. - self.threshold)
        threshold = self.threshold

        # Vertex parameter
        v0 = torch.FloatTensor(mesh.vertices).to(self.device)
        v = torch.nn.Parameter(v0.clone())

        # Faces of mesh
        faces = torch.LongTensor(mesh.faces).to(self.device)

        # Start optimization
        optimizer = optim.RMSprop([v], lr=1e-4)

        for it_r in trange(self.refinement_step):
            optimizer.zero_grad()

            # Loss
            face_vertex = v[faces]
            eps = np.random.dirichlet((0.5, 0.5, 0.5), size=faces.shape[0])
            eps = torch.FloatTensor(eps).to(self.device)
            face_point = (face_vertex * eps[:, :, None]).sum(dim=1)

            face_v1 = face_vertex[:, 1, :] - face_vertex[:, 0, :]
            face_v2 = face_vertex[:, 2, :] - face_vertex[:, 1, :]
            face_normal = torch.cross(face_v1, face_v2)
            face_normal = face_normal / \
                (face_normal.norm(dim=1, keepdim=True) + 1e-10)
            face_value = torch.sigmoid(
                self.model.decode(face_point.unsqueeze(0), z, c).logits
            )
            normal_target = -autograd.grad(
                [face_value.sum()], [face_point], create_graph=True)[0]

            normal_target = \
                normal_target / \
                (normal_target.norm(dim=1, keepdim=True) + 1e-10)
            loss_target = (face_value - threshold).pow(2).mean()
            loss_normal = \
                (face_normal - normal_target).pow(2).sum(dim=1).mean()

            loss = loss_target + 0.01 * loss_normal

            # Update
            loss.backward()
            optimizer.step()

        mesh.vertices = v.data.cpu().numpy()

        return mesh